In [2]:
import netCDF4 as nc
import datetime

HOST = '/Users/jonsmi/OneDrive - NERC/Documents/BAS_RoutePlanner/Jan_2022/Jan_10th_Development/RoutePlanner/resources/'

iceDataPath     = '{}/bsose_i122_2013to2017_1day_SeaIceArea.nc'.format(HOST)
currentDataPath = '{}/SOSE_surface_velocity_6yearMean_2005-2010.nc'.format(HOST)

ds = nc.Dataset(iceDataPath)

In [3]:
timeindx = 515
time = ds['time'][timeindx]
epoch = datetime.datetime(2012, 12, 1)
time = epoch + datetime.timedelta(0, int(time))

print(time.strftime("%Y-%m-%d"))


2014-05-31


In [5]:
class IcePoint:
    def __init__(self, time, long, lat, iceArea, depth):
        self.time = time
        self.long = long
        self.lat = lat
        self.iceArea = iceArea
        self.depth = depth
        
    def toJSON(self):
        s = "{"
        s += "\"time\": \"" + str(self.time) + "\","
        s += "\"long\":" + str(self.long) + ","
        s += "\"lat\":" + str(self.lat) + ","
        s += "\"iceArea\":" + str(self.iceArea) + ","
        s += "\"depth\":" + str(self.depth)
        s += "}"
        return s
    
    def to_dict(self):
        return {
            'time': self.time,
            'long': self.long,
            'lat': self.lat,
            'iceArea': self.iceArea,
            'depth' : self.depth,
        }

In [6]:
#Constructs list of IcePoint objects for a single frame, specifed by 'timeindx'

timeindx = 515 #Set timeindx to select which day to generate data for.


longindx = 0
latindx = 0

epoch = datetime.datetime(2012, 12, 1)
icePoints = []

for longindx in range(len(ds['XC'])):
    for latindx in range(len(ds['YC'])):
        
        # read icePoint from datesouce ds.
        time = ds['time'][timeindx]
        time = epoch + datetime.timedelta(0, int(time)) # convert to datetime
        long = ds['XC'][longindx]
        lat = ds['YC'][latindx]
        iceArea = ds['SIarea'][timeindx, latindx, longindx]
        depth = ds['Depth'][latindx, longindx]
        
        #TODO change to better represent missing values
        if str(iceArea) == "nan":
            iceArea = 0
        
        
        # create IcePoint object
        icePoint = IcePoint(time, long, lat, iceArea, depth)
        # append to list of all icePoints
        icePoints.append(icePoint)

print("Done")

KeyboardInterrupt: 

In [29]:
#construct json from list of icepoints
json = "{ \"icepoints\":["

for icepoint in icePoints:
    json += icepoint.toJSON() + ",\n"
    
json = json[:-2] # remove last comma and newline
json += "]}"
print("Done")

Done


In [30]:
#Format time
time = ds['time'][timeindx]
epoch = datetime.datetime(2012, 12, 1)
time = epoch + datetime.timedelta(0, int(time))
time = time.strftime("%Y-%m-%d")

#Write list of icepoints to json.
filename = "../resouces/icepoints" + time + ".json"
f = open(filename, "x")
f.write(json)
f.close()